### Creating Spark session
Spark session is "the gateway to the structured data processing".
It can be used to create datasets, dataframes, user defined functions and execute SQL.
It replaces SQLContext used in previous versions of Apache Spark.

In [1]:
import sagemaker_pyspark
from pyspark.sql import SparkSession

# This enables s3 support in Spark. You may need to restart the kernel!
classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = SparkSession.builder \
    .master("local") \
    .appName("Teemuko mle") \
    .config("spark.driver.extraClassPath", classpath) \
    .getOrCreate()

### Loading the CSV-data from S3

In [12]:
#filePath = "s3a://mle7-data/train_numeric.csv"
#filePath = "s3a://sagemaker-tmukoo/ratings-5000.csv"
filePath = "s3a://sagemaker-tmukoo/ratings.csv"


ratings = spark.read.load(filePath, format="csv", inferSchema="true", header="true")
#df=spark.read.csv(filePath,header=True)

In [13]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [14]:

ratings.describe().show()

+-------+-----------------+------------------+------------------+--------------------+
|summary|           userId|           movieId|            rating|           timestamp|
+-------+-----------------+------------------+------------------+--------------------+
|  count|         26024289|          26024289|          26024289|            26024289|
|   mean| 135037.090248114|15849.109677040553|3.5280903543608817|1.1712584326913223E9|
| stddev|78176.19722170963|31085.257531391508|1.0654427636662405|2.0528887028185263E8|
|    min|                1|                 1|               0.5|           789652004|
|    max|           270896|            176275|               5.0|          1501829870|
+-------+-----------------+------------------+------------------+--------------------+



In [15]:
# How many distinct userIds?
ratings.select('userId').distinct().count()

270896

In [16]:
# How many distinct movieIds?
ratings.select('movieId').distinct().count()

45115

In [17]:
from pyspark.sql.functions import mean, min, max
ratings.select([mean('rating'), min('rating'), max('rating')]).show()

+------------------+-----------+-----------+
|       avg(rating)|min(rating)|max(rating)|
+------------------+-----------+-----------+
|3.5280903543608817|        0.5|        5.0|
+------------------+-----------+-----------+



In [18]:
ratings.take(3)

[Row(userId=1, movieId=110, rating=1.0, timestamp=1425941529),
 Row(userId=1, movieId=147, rating=4.5, timestamp=1425942435),
 Row(userId=1, movieId=858, rating=5.0, timestamp=1425941523)]

### Machine learning to the rescue!
Note! SparkML will eventually replace MLlib.

Example: https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

In [ ]:
# DON'T! from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

#lines = spark.read.text("data/mllib/als/sample_movielens_ratings.txt").rdd
#parts = lines.map(lambda row: row.value.split("::"))
#ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=long(p[3])))
#ratings = spark.createDataFrame(ratingsRDD)

(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

In [ ]:
userRecs.take(3)

In [ ]:
movieRecs.take(3)